In [1]:
import sources.logins as lg
from sqlite3 import connect
import sources.parts_list_0725 as pl
import pandas as pd
from datetime import datetime as dt
import xlwings as xw

In [2]:
conn = connect('quotation.db')


In [3]:
pr110 = xw.Book('New ML/PR110_Stock_in_Aug_08082023.xlsx')


In [4]:
sh = pr110.sheets[0]
part_list = []
for i in range(16,1000):
#     print(i)
    try:
        if sh.range(f'C{i}').value.startswith('FFVN'):
            info = sh.range(f'C{i}').value[6:-1]
        if sh.range(f'C{i}').value.startswith('Stock'):
            info = 'Stock'
#         print(info)
    except Exception as e:
        print (i,e)
    if sh.range(f'D{i}').value != None: 
        print(i,sh.range(f'D{i}').value.strip(),info.strip(),sh.range(f'F{i}').value)
        part_list.append(sh.range(f'D{i}').value.strip())
    if sh.range(f'D{i}').value == 'Reviewed by':
        print(f'Stop {i}')
        break
        
parts = tuple(part_list)

17 858Y200030D EC-600WI, 1C692K656 1.0
18 343A1295408G EC-600WI, 1C692K656 1.0
21 370Y200247B EG-720R, 1G412K168 1.0
24 898Y201120C EG-720R, 1G412K193 1.0
25 370Y200247B EG-720R, 1G412K193 1.0
26 830Y200192E EG-720R, 1G412K193 1.0
29 57B1251017A ED-530XT, RD102A106 1.0
30 68B11145370 ED-530XT, RD102A106 1.0
31 68A11029950 ED-530XT, RD102A106 1.0
32 53B11413710 ED-530XT, RD102A106 1.0
33 343A1295408G ED-530XT, RD102A106 1.0
34 343A1331613D ED-530XT, RD102A106 1.0
35 68B9893902 ED-530XT, RD102A106 1.0
38 68B11145370 ED-530XT8, RD103A589 1.0
39 370Y200098 ED-530XT8, RD103A589 1.0
40 10A1235703B ED-530XT8, RD103A589 1.0
41 68B9893902 ED-530XT8, RD103A589 1.0
42 343A1295408G ED-530XT8, RD103A589 1.0
43 343A1331613D ED-530XT8, RD103A589 1.0
44 33B10212171 ED-530XT8, RD103A589 1.0
45 48B10212161 ED-530XT8, RD103A589 1.0
46 53B1021214A ED-530XT8, RD103A589 4.0
49 68A1251518J EC-530WI, 1C603K040 1.0
50 343A1295408G EC-530WI, 1C603K040 1.0
51 113Y200432B EC-530WI, 1C603K040 1.0
52 113Y200431B EC

In [ ]:
part_list = '''
375N200200B
375N100282A
340Y200004A
343A1337437B
68A921163H
898Y201120C
375N200051
375N100275A
342A979721F
342A1318160N
68A1251518J
343A1295408G
113Y200432B
113Y200431B
343A1331615E

'''
parts = tuple(part_list.strip().split('\n'))
parts 


In [5]:
pr110.sheets.add('Sheet1',after=sh.name)
sh1 = pr110.sheets('Sheet1')

In [6]:
# parts = ('858Y200030D','858Y200030D')
q = f'''
        SELECT [Parts Name(EN)] AS [PART_DESCRIPTION],PN.VIE,
            CASE 
                WHEN substr(P.[Article Code],4,1) IN ('Y','N','S') THEN 'F'
                WHEN substr(P.[Article Code],1,1) = 'J' THEN 'F'
                ELSE 'FW12G' END AS SAP,
            P.[Article Code] as part_num,p.[Subsidiaries (U$)] AS[Sub Price],p.[Cost of Goods sold] as [FFVN Price],
            p.[Dealer Standard Price (wiout VAT)] as [Dealer Price]



            FROM prices P
            LEFT JOIN part_name pn ON TRIM(REPLACE(p.[Parts Name(EN)],'ROHS2','')) = pn.part_description
            WHERE part_num IN {parts}
        
     '''

parts_list = pd.read_sql(q,conn)
for part in parts:
    if part not in list(parts_list['part_num']):print(part)
parts_list

342A979721F
Reviewed by


,PART_DESCRIPTION,Vie,SAP,part_num,Sub Price,FFVN Price,Dealer Price
0,LIGHT CONTOROL PCB ROHS,None,F,113Y101074B,186,197.16,334.169492
1,NT TGCONT ROHS2,None,F,113Y200431B,155,164.30,278.474576
2,NT INTERFACE ROHS2,None,F,113Y200432B,93,98.58,167.084746
3,LR LOCK KNOB ROHS2,Khóa điều khiển,F,340Y200004A,2,2.12,3.593220
4,FCT PIPE ROHS,None,F,369N130049A,18,19.08,32.338983
5,FORCEPS CHANNEL TUBE(D94A)ROHS,None,F,370Y200098,109,115.54,195.830508
6,FSA-G412A ROHS,None,F,370Y200247B,264,279.84,474.305085
7,A/W TUBE ROHS,None,F,370Y200313,171,181.26,307.220339
8,AW-TUBE FG391A ROHS2,None,F,370Y200592,67,71.02,120.372881
9,FCT-D126A ROHS2,Kênh sinh thiết,F,370Y200989B,589,624.34,1058.203390


In [ ]:
part = '342A979721E'
part_list.append(part)
parts = tuple(part_list)

In [ ]:
part_src = parts_list[parts_list['part_num']=='342A979721D']
part_src = part_src.reset_index()
part_src

In [7]:
# add info to sheet[1]
i_count = 0
po_name = 'FFVN-PO-110'
for i in range(16,1000):
    try:
        if sh.range(f'C{i}').value.startswith('FFVN'):
            info = sh.range(f'C{i}').value[6:-1]
        if sh.range(f'C{i}').value.startswith('Stock'):
            info = 'Stock'
#         print(info)
    except Exception as e:
        print (i,e)
#     print(i)
    if sh.range(f'D{i}').value != None: 
#         print(i,sh.range(f'D{i}').value.strip(),info.strip())
        part_number = sh.range(f'D{i}').value.strip()
        sh1.range('A5').offset(i_count).value = i_count + 1
        sh1.range('D5').offset(i_count).value = i_count + 1
        sh1.range('H5').offset(i_count).value = info
        sh1.range('I5').offset(i_count).value = sh.range(f'F{i}').value
        sh1.range('C5').offset(i_count).value = po_name
        try:
            part_src = parts_list[parts_list['part_num']==part_number]
            part_src = part_src.reset_index()




            sh1.range('E5').offset(i_count).value = part_src['SAP'][0]
            sh1.range('F5').offset(i_count).value = part_src['part_num'][0]
            sh1.range('G5').offset(i_count).value = part_src['PART_DESCRIPTION'][0]

            sh1.range('J5').offset(i_count).value = part_src['Sub Price'][0]
        except:
            sh1.range('F5').offset(i_count).value = part_number
            
        i_count +=1
    if sh.range(f'D{i}').value == 'Reviewed by':
        print('Done')
        break

92 'NoneType' object has no attribute 'startswith'
94 'NoneType' object has no attribute 'startswith'
95 'NoneType' object has no attribute 'startswith'
96 'NoneType' object has no attribute 'startswith'
97 'NoneType' object has no attribute 'startswith'
98 'NoneType' object has no attribute 'startswith'
99 'NoneType' object has no attribute 'startswith'
Done


In [ ]:
part_number